# **Quickstart de BPETokenizer**

¡Bienvenido/a al inicio rápido de `BPETokenizer`!

En este notebook, realizaremos un recorrido práctico por las funcionalidades clave de la librería. Aprenderás a:
1.  **Instalar** la librería directamente desde GitHub.
2.  **Entrenar** un tokenizador BPE desde cero utilizando un corpus de texto (usaremos *El Quijote*).
3.  **Guardar** el tokenizador entrenado para usarlo más tarde.
4.  **Cargar** y **utilizar** el tokenizador para codificar (tokenizar) y decodificar texto.

Este es el lugar perfecto para ver el código en acción. Para una explicación teórica detallada de cómo funciona el algoritmo por dentro, no olvides consultar el `README.md` y el notebook `explicación_bpe.ipynb` en el repositorio.

¡Empecemos!

## **1. Instalación del Proyecto**

El primer paso es instalar la librería `BPETokenizer`. Como este notebook se ejecuta en un entorno limpio (como Google Colab), necesitamos descargarla. El siguiente comando la instala directamente desde su repositorio de GitHub.

In [ ]:
!pip install git+https://github.com/ErikSarriegui/BPETokenizer

## **2. Entrenamiento de un Nuevo Tokenizador**

Ahora que tenemos la librería instalada, vamos a entrenar nuestro propio tokenizador. El proceso consta de los siguientes pasos:

1.  **Obtener un corpus de texto**: Para entrenar un tokenizador, necesitamos una buena cantidad de texto. Usaremos la librería `requests` para descargar el texto completo de *Don Quijote de la Mancha*, que utilizaremos como corpus en Español.
2.  **Crear un `BPETrainer`**: Esta es la clase encargada de gestionar el proceso de entrenamiento.
3.  **Llamar al método `.train()`**: Aquí es donde ocurre la magia. Le pasamos el texto y definimos el tamaño del vocabulario que queremos crear.
    *   `vocab_size=270`: Este es el tamaño total del vocabulario. Como partimos de 256 tokens base (un token por cada byte), esto significa que realizaremos `270 - 256 = 14` fusiones. Es un vocabulario muy pequeño, ideal para una demostración rápida.
    *   `verbose=True`: Nos permite ver en tiempo real cada una de las 14 fusiones que el algoritmo va aprendiendo. ¡Es perfecto para entender el proceso!
4.  **Guardar el tokenizador**: Una vez entrenado, usamos `.save()` para guardar las reglas de fusión y el vocabulario en un directorio. En este caso, lo llamaremos `tokenizer/`.

In [ ]:
from BPETokenizer import BPETrainer, BPETokenizer
import requests

# 1. Obtener el corpus de texto
print("Descargando el texto de El Quijote...")

text = requests.get("https://babel.upm.es/~angel/teaching/pps/quijote.txt").text
print(f"Texto descargado. Longitud: {len(text)} caracteres.\n")

# 2. Crear el entrenador
trainer = BPETrainer()

# 3. Entrenar el tokenizador
print("Iniciando el entrenamiento del tokenizador...")
trainer.train(
    text = text,
    vocab_size = 270,  # 256 bytes base + 14 fusiones
    verbose = True
)
print("\n¡Entrenamiento completado!")

# 4. Guardar el tokenizador
trainer.save("tokenizer")
print("Tokenizador guardado en la carpeta 'tokenizer/'.")

## **3. Uso del Tokenizador (Inferencia)**

¡Nuestro tokenizador ya está entrenado y guardado! Ahora vamos a cargarlo y a usarlo para lo que fue creado: convertir texto en secuencias de números (tokens) y viceversa.

1.  **Cargar el tokenizador**: Usamos `BPETokenizer.from_dir()` para cargar la configuración que guardamos en la carpeta `tokenizer/`.
2.  **Pre-tokenizar (`.split()`)**: Antes de la tokenización BPE, el texto se divide usando una expresión regular (regex). Este paso separa palabras, puntuación y espacios. El método `.split()` nos permite ver cómo queda el texto después de esta división inicial.
3.  **Codificar (`.encode()`)**: Este es el proceso de tokenización principal. Convierte un string de texto en una lista de IDs de tokens numéricos, aplicando las reglas de fusión que aprendió durante el entrenamiento.
4.  **Decodificar (`.decode()`)**: Es el proceso inverso. Toma una lista de IDs de tokens y la reconstruye en un string de texto legible por humanos.
5.  **Verificar la consistencia**: Finalmente, comprobamos que `decode(encode(texto))` nos devuelve el texto original. Esta es una prueba clave para asegurar que nuestro tokenizador funciona correctamente y no pierde información.

In [ ]:
# Texto de ejemplo para probar
text = "Hola mundo"
print(f"Texto original: '{text}'\n")

# 1. Cargar el tokenizador
tokenizer = BPETokenizer.from_dir("tokenizer")

# 2. Ver la pre-tokenización (división por regex)
splited_text = tokenizer.split(text)
print(f"Texto pre-tokenizado (split): {splited_text}")

# 3. Codificar el texto a tokens
tokenized_text = tokenizer.encode(text)
print(f"Texto codificado (tokens): {tokenized_text}")

# 4. Decodificar los tokens a texto
# Usaremos una lista de ejemplo para ver el proceso
encoded_text = [72, 111, 108, 97, 32, 109, 117, 110, 100, 111] # 'Hola mundo' en bytes UTF-8
decoded_text = tokenizer.decode(encoded_text)
print(f"Texto decodificado: '{decoded_text}'")

# 5. Verificación de consistencia (ida y vuelta)
is_consistent = tokenizer.decode(tokenizer.encode(text)) == text
print(f"\n¿La decodificación de la codificación devuelve el texto original? -> {is_consistent}")